In [15]:
import torch
from torch.utils.data import DataLoader
from engine.datasets import GraphDataset
from engine.Tmodel import GraphTransformerV2
from engine.train import train_model
from engine.predict import predict
from engine.cr_boosters.adam import Adam
import engine.core_rec as cr
import numpy as np
import networkx as nx
import engine.vish_graphs as vg
# from ca_videos import load_ca_videos_dataset  # Assuming you have a module to load CA Videos dataset
import pandas as pd


In [16]:
try:
    ca_videos_data = pd.read_csv('nanodata.csv')  # Load your dataset
except pd.errors.ParserError as e:
    print(f"Error loading data: {e}")
    # Handle the error or clean the data file

# Create edges based on videos from the same channel
edges = []
channels = ca_videos_data['channel_title'].unique()
for channel in channels:
    channel_videos = ca_videos_data[ca_videos_data['channel_title'] == channel]
    video_ids = channel_videos['video_id'].values
    for i in range(len(video_ids)):
        for j in range(i + 1, len(video_ids)):
            edges.append((video_ids[i], video_ids[j]))

G = nx.Graph()
G.add_edges_from(edges)
adj_matrix = nx.to_numpy_array(G)
weight_matrix = np.ones_like(adj_matrix)  # Example weight matrix with all ones

graph_dataset = GraphDataset(adj_matrix, weight_matrix)
data_loader = DataLoader(graph_dataset, batch_size=32, shuffle=True)


Error loading data: Error tokenizing data. C error: EOF inside string starting at row 922


In [17]:

# Step 2: Initialize the model
model = GraphTransformerV2(num_layers=2, d_model=128, num_heads=4, d_feedforward=512, input_dim=adj_matrix.shape[1])


In [18]:
# Step 3: Initialize the optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# Step 4: Train the model using engine.core_rec
criterion = torch.nn.MSELoss()
cr.train_model(model, data_loader, criterion, optimizer, num_epochs=10)




# Step 5: Make predictions
node_index = 11 # Example node index
recommended_nodes = cr.predict(model, adj_matrix, node_index, top_k=5, threshold=0.5)
print(f"Recommended nodes for node {node_index}: {recommended_nodes}")

# Step 6: Draw the graph
vg.draw_graph_3d(adj_matrix, recommended_nodes=recommended_nodes, transparent_labeled=False, edge_weights=weight_matrix)


Epoch 1/10, Loss: 0.10065151005983353
Epoch 2/10, Loss: 0.09938107430934906


KeyboardInterrupt: 

In [12]:

# Step 3: Initialize the optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# Step 4: Train the model
criterion = torch.nn.MSELoss()
train_model(model, data_loader, criterion, optimizer, num_epochs=10)


Epoch 1/10, Loss: 0.10021856427192688


KeyboardInterrupt: 

In [ ]:

# Step 5: Make predictions
node_index = 1147  # Example node index
recommended_nodes = cr.predict(model, adj_matrix, node_index, top_k=5, threshold=0.5)
print(f"Recommended nodes for node {node_index}: {recommended_nodes}")

# Step 6: Draw the graph
vg.draw_graph_3d(adj_matrix, recommended_nodes=recommended_nodes, transparent_labeled=False, edge_weights=weight_matrix)